In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Building the data set

In [13]:
import deeplenstronomy.deeplenstronomy as dl

config_file_name = 'config.yaml' # name of your file
dataset = dl.make_dataset(config_file_name)

# Convert the images in a tensor
dataset_tensor = torch.from_numpy(dataset.CONFIGURATION_1_images)

In [18]:
type(dataset.CONFIGURATION_1_metadata.iloc[0]['OBJID-g'])

numpy.int64

In [38]:
for ID_imgs in dataset.CONFIGURATION_1_metadata[['OBJID-g']].values:
    file_name = "DeeplenstronomyDataset/"+str(ID_imgs[0])+".csv"
    file_array = dataset.CONFIGURATION_1_images[ID_imgs].ravel()
    np.savetxt(file_name, file_array, delimiter=",")

In [48]:
from torch.utils.data import Dataset, DataLoader
import os

class CombineDataset(Dataset):

    def __init__(self, frame, id_col, label_name, path_imgs, nb_channel, transform=None):
        """
        Args:
            frame (pd.DataFrame): Frame with the tabular data.
            id_col (string): Name of the column that connects image to tabular data
            label_name (string): Name of the column with the label to be predicted
            path_imgs (string): Path to the folder where the images are.
            nb_channel (int): Number of channels
            transform (callable, optional): Optional transform to be applied
                on a sample, you need to implement a transform to use this.
        """
        self.frame = frame
        self.id_col = id_col
        self.label_name = label_name
        self.path_imgs = path_imgs
        self.nb_channel = nb_channel
        #self.transform = transform

    def __len__(self):
        return (self.frame.shape[0])

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        #complete image path and read
        img_name = self.frame[self.id_col].iloc[idx]
        path = os.path.join(self.path_imgs,img_name)
        image = read_dicom_image(path)
        image = image.reshape(self.nb_channel, 64, 64)
        image = torch.from_numpy(image.astype(np.float32))

        #get the other features to be used as training data
        feats = [feat for feat in self.frame.columns if feat not in [self.label_name,self.id_col]]
        feats  = np.array(self.frame[feats].iloc[idx])
        feats = torch.from_numpy(feats.astype(np.float32))
       
        
        #get label
        label = np.array(self.frame[self.label_name].iloc[idx])
        label = torch.from_numpy(label.astype(np.float32))
        #label = torch.as_tensor(np.array(label).astype('float'))

        return image, feats, label

In [46]:
train_set = CombineDataset(dataset.CONFIGURATION_1_metadata,'OBJID-g','FVC',"DeeplenstronomyDataset/",2)

loader_trainer = DataLoader(
    train_set,
    batch_size = 2,
    shuffle = True,
    num_workers = 0,
    drop_last=True
)

In [49]:
for data in loader_trainer:
    x,z, y = data
    
    print('Image Loaded Size = ', x.shape)
    print('Tabular data = ', z)
    print('Label = ', y)

TypeError: join() argument must be str or bytes, not 'int64'

## Building neural network

In [3]:
classes = ('NoError', 'MassError', 'SourceError')

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(5, 7, 6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(7, 16, 6)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.pool(nn.SELU(self.conv1(x)))
        x = self.pool(nn.SELU(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = nn.SELU(self.fc1(x))
        x = nn.SELU(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)